Processo de Scrapy

In [ ]:
import os
import subprocess

def execute_commands(comando):
    retorno = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    if retorno.returncode == 0:
        print(f"Comando '{comando}' executado com sucesso!")
    else:
        print(f"Erro na execução de: {comando}")
        print(retorno.stderr.decode())

# acessar diretorio do projeto
os.chdir("..\scrapcards")

# comandos para executar respectivos spiders
comandos = ["scrapy crawl ligamagic -O datasets/mtg.json", "scrapy crawl ligapoke -O datasets/ptcg.json", "scrapy crawl ligaygo -O datasets/ygo.json"]

for comando in comandos:
    execute_commands(comando)


O banco de dados foi preparado de maneira que a coluna "acronym" da tabela "game"corresponda
ao nome do arquivo JSON sem a extenção, o que permite a busca correta pelo ID do jogo. Uma 
ez que temos o ID, é iniciado o processo de inserção de coleções e suas cartas.

In [2]:
import psycopg2
import json
import os

# Variáveis banco de dados
host = "localhost"
database = "ligaCards"
user = "postgres"
password = "admin"
conn = psycopg2.connect(host=host, database=database, user=user, password=password)
cursor = conn.cursor()

# Buscando os nomes dos arquivos json
folder = '..\scrapcards\datasets'
files_json = [arquivo for arquivo in os.listdir(folder) if arquivo.endswith('.json')]

# Iterando sobre os arquivos JSON
for file in files_json:
    path_json = os.path.join(folder, file)
    game_acronym = os.path.splitext(file)[0]
    with open(path_json, encoding='utf-8') as file:
        data = json.load(file)

    cursor.execute(f"SELECT id FROM game WHERE LOWER(acronym) LIKE '%{game_acronym}%'")
    game_id = cursor.fetchone()

    for iten in data:
        try:
            query = "INSERT INTO collection (game_id, name, acronym, cards_quantity, lowest_price, average_price, highest_price) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            values = (game_id[0], iten['name'].replace("'", ""), iten['acronym'], iten['cards_quantity'], iten['lowest_price'], iten['avarege_price'], iten['highest_price'])
            cursor.execute(query, values)
            conn.commit()
        except Exception as e:
            print(e)
            print(values)
            break

        cursor.execute(f"""SELECT id FROM collection WHERE name = '{iten['name'].replace("'", "")}'""")
        collection_id = cursor.fetchone()

        for card in iten['cards']:
            try:
                query = "INSERT INTO card(collection_id, lowest_price, highest_price, name_en, name_pt) VALUES (%s, %s, %s, %s, %s)"
                values = (collection_id[0], card['precoMenor'], card['precoMaior'], card['nEN'].replace("'", ""), card['nPT'].replace("'", ""))
                cursor.execute(query, values)
                conn.commit()
            except Exception as e:
                print(e)
                print(values)
                break
            

cursor.close()
conn.close()
